<a href="https://colab.research.google.com/github/umarmohammed/measuring-bias-in-a-dataset/blob/project-4/4.%20Recompute%20bias%20metrics%20on%20transformed%20data.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
# Mount Google Drive locally
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [2]:
!pip install aif360

     |████████████████████████████████| 174kB 6.1MB/s 


In [7]:
# load the BinaryLabelDataset
from joblib import load

(dataset_transformed, dataset_pred_transformed) = load("/content/drive/MyDrive/liveProject/mortgate_data_wpredict_bld_transformed.pkl.gz")

type(dataset_transformed)

aif360.datasets.binary_label_dataset.BinaryLabelDataset

In [8]:
from aif360.metrics import ClassificationMetric

privileged_group = [{'applicant_sex_name_Female': 0}] 
unprivileged_group = [{'applicant_sex_name_Female': 1}]

metric = ClassificationMetric(dataset_transformed, dataset_pred_transformed, unprivileged_group, privileged_group)

fair_metrics = {"AvgOddsDiff": metric.average_odds_difference, "DispImpact": metric.disparate_impact, "EqualOppDiff": metric.equal_opportunity_difference, "StatParity": metric.statistical_parity_difference }

metrics = [[ff, fair_metrics[ff]()] for ff in fair_metrics]

import pandas as pd 
pd.DataFrame(metrics, columns=["Metric", "Value"])

,Metric,Value
0,AvgOddsDiff,0.000000
1,DispImpact,1.138932
2,EqualOppDiff,0.000000
3,StatParity,0.066741
